In [1]:
# '''
# Goto config ->config.yaml
# Paste below code

# model_evaluation:
#   root_dir: artifacts/model_evaluation
#   data_path: artifacts/data_transformation/samsum_dataset
#   model_path: artifacts/model_trainer/pegasus-samsum-model
#   tokenizer_path: artifacts/model_trainer/tokenizer
#   metric_file_name: artifacts/model_evaluation/metrics.csv
# '''

In [2]:
import os

In [3]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Project\\Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment'

export MLFLOW_TRACKING_URI=https://dagshub.com/SunilKumar-ugra/Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment.mlflow

export MLFLOW_TRACKING_USERNAME=SunilKumar-ugra

export MLFLOW_TRACKING_PASSWORD=ff039c8b90d721cb1c7f99b788c8bfe02f4d345b

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SunilKumar-ugra/Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SunilKumar-ugra"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ff039c8b90d721cb1c7f99b788c8bfe02f4d345b"

# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    all_params:dict
    model_path: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri:str  

# Config folder 
1. Copy get_model_evaluation_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import ModelEvaluationConfig from wine_quality.entity using this line  

from wine_quality.entity import ModelEvaluationConfig:


In [8]:
from wine_quality.constants import *
from wine_quality.utils import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column =schema.name,
            mlflow_uri="https://dagshub.com/SunilKumar-ugra/Wine_Quality_MLflow_AWS_CICD_Pipeline_End_to_End_Deployment.mlflow"
           )

        return model_evaluation_config

# Component folder
1. Copy entire ModelEvaluation class
2. Goto Component folder [ src-> textSummarizer -> components - > model_evaluation.py ]
2. Paste ModelEvaluation class in model_evaluation.py
3. Import model_evaluation_config from entity using this line 

from wine_quality.entity import model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from wine_quality.utils import save_json

C:\Users\This PC\AppData\Roaming\Python\Python38\site-packages\pydantic\_internal\_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\This PC\AppData\Roaming\Python\Python38\site-packages\pydantic\_internal\_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

# Pipeline folder

1. Goto Component folder [ src-> textSummarizer -> pipeline - > stage_05_model_evaluation.py ]

2. Paste below code in stage_05_model_evaluation.py

In [12]:
''' 

from wine_quality import logger
from wine_quality.config. import ConfigurationManager
from wine_quality.components.model_evaluation import ModelEvaluation
class ModelTrainerTrainingPipeline:
    def __init__(self):
        pass
    def main(self):
'''

' \n\nfrom wine_quality import logger\nfrom wine_quality.config. import ConfigurationManager\nfrom wine_quality.components.model_evaluation import ModelEvaluation\nclass ModelTrainerTrainingPipeline:\n    def __init__(self):\n        pass\n    def main(self):\n'

3. Copy try block code 
4. Paste in main function in stage_05_model_evaluation.py

5. Import ConfigurationManager from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import ModelTrainer  from  components.model_trainer  using this line  
from wine_quality.components..model_evaluation  import class ModelEvaluation:


In [13]:
from wine_quality import logger

In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

[2023-09-18 00:31:49,803] - wine_quality - INFO - __init__ file - LineNum:27   - yaml file : config\config.yaml loaded successfully
[2023-09-18 00:31:49,810] - wine_quality - INFO - __init__ file - LineNum:27   - yaml file : params.yaml loaded successfully
[2023-09-18 00:31:49,819] - wine_quality - INFO - __init__ file - LineNum:27   - yaml file : schema.yaml loaded successfully
[2023-09-18 00:31:49,826] - wine_quality - INFO - __init__ file - LineNum:43   - Created Directory At: artifacts
[2023-09-18 00:31:49,829] - wine_quality - INFO - __init__ file - LineNum:43   - Created Directory At: artifacts/model_evaluation


[2023-09-18 00:31:51,967] - wine_quality - INFO - __init__ file - LineNum:55   - JSON File Saved At artifacts\model_evaluation\metrics.json


Successfully registered model 'ElasticnetModel'.
2023/09/18 00:32:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


In [ ]:
'''
Example
from wine_quality.config import ConfigurationManager
from wine_quality.components.model_evaluation import ModelEvaluation
from wine_quality import logger




class ModelEvaluationTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        model_evaluation_config = config.get_model_evaluation_config()
        model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
        model_evaluation_config.evaluate()
'''

In [ ]:
'''

1. Goto main.py and paste below code

STAGE_NAME = "Model Evaluation stage"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_evaluation = ModelEvaluationTrainingPipeline()
   model_evaluation.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e

2. Run main.py 
        
'''